https://huggingface.co/transformers/quickstart.html

https://github.com/huggingface/transformers/tree/master/examples/language-modeling

https://github.com/huggingface/transformers/blob/master/src/transformers/training_args.py

In [1]:
import os
import json
from glob import glob
import numpy as np
from transformers import (
    GPT2Tokenizer, 
    GPT2LMHeadModel, 
    TextDataset, 
    PreTrainedTokenizer, 
    DataCollatorForLanguageModeling
)

In [2]:
gpt2pretrainedstr = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(gpt2pretrainedstr)
model = GPT2LMHeadModel.from_pretrained(gpt2pretrainedstr)

In [3]:
input_path = 'finetune/data/suncoasthost/'
#file_path_train = os.path.join(input_path, 'train.txt')
file_path_valid = os.path.join(input_path, 'valid.txt')
file_path_eval = os.path.join(input_path, 'test.txt')

In [4]:
def get_dataset(file_path, tokenizer: PreTrainedTokenizer):
    return TextDataset(
        tokenizer=tokenizer, 
        file_path=file_path, 
        block_size=tokenizer.max_len, 
        overwrite_cache=True,
    )

In [ ]:
#train_dataset = get_dataset(file_path_train, tokenizer=tokenizer)
train_dataset = get_dataset(file_path_valid, tokenizer=tokenizer)
eval_dataset = get_dataset(file_path_eval, tokenizer=tokenizer)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False,
)

In [5]:
from transformers import TrainingArguments, Trainer

In [6]:
training_args = TrainingArguments(
    output_dir='finetune/model/suncoasthost',
    do_train=True,
    do_eval=True,
)

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    prediction_loss_only=True,
)

In [ ]:
trainer.train()
trainer.save_model()